In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [34]:
train = pd.read_csv(r'E:\data\kaggle\spookyauthor\train.csv',index_col=False)
test = pd.read_csv(r'E:\data\kaggle\spookyauthor\test.csv',index_col=False)

In [35]:
train.text += '\\'
#import unidecode
#decoder = unidecode.unidecode
train.text = train.text.str.lower() #apply(decoder)
t0 = train[train.author==train.author.unique()[0]].text
t0.reset_index(drop=True,inplace=True)

In [39]:
text = train.text.sum()
text += test.text.str.lower().sum()
chars = sorted(list(set(text)))
print(chars,len(chars))

[' ', '"', "'", ',', '.', ':', ';', '?', '\\', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'â', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ñ', 'ô', 'õ', 'ö', 'û', 'ü', 'α', 'δ', 'ν', 'ο', 'π', 'ς', 'υ', 'ἶ'] 61


In [52]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

input_size = len(chars)
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 1
sentences = []
next_chars = []
def linetotensor(line):
    l = len(line)
    line += '\\'
    sentence = line[: l-1]
    next_char = line[1: l]
    x = np.zeros((l, 1, input_size))
    y = np.zeros((l,1))
    for t,letter in enumerate(sentence):
        x[t, 0, char_indices[letter]] = 1
    for t,letter in enumerate(next_char):
        y[t,0 ] = char_indices[letter]
    return x,y

(232, 1, 61)

In [ ]:
tf.reset_default_graph()

epoch_num = 20
sample_size = len(t0)
batch_size = 1
num_step = 4
rnn_size = 256



rnn = tf.contrib.rnn.BasicLSTMCell(rnn_size,activation=tf.tanh)


#input and label

X = tf.placeholder(tf.float32,shape=[None,batch_size,input_size])
Y = tf.placeholder(tf.int32,shape=[None,batch_size])


W = tf.Variable(tf.random_normal([rnn_size,input_size]))
b = tf.Variable(tf.random_normal([input_size])) 

initial_state = rnn.zero_state(batch_size, dtype=tf.float32)
output,state = tf.nn.dynamic_rnn(rnn,X,
                                 initial_state=initial_state,
                                time_major=True)

logits = tf.tensordot(output,W,[[2],[0]])+b
prob = tf.nn.softmax(logits)
loss = tf.losses.sparse_softmax_cross_entropy(labels = Y, logits = logits)
LOSS = tf.reduce_sum(loss)
LOSS_ = tf.reduce_mean(loss)
      

session =tf.Session()
session.run(tf.global_variables_initializer())
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(LOSS)
for epoch in range(epoch_num):
    
    for i in range(sample_size//batch_size):
        x,y = linetotensor(t0[i])
        
        #'''
        
        
        a = session.run(train_step,feed_dict={X:x,Y:y})
        current_loss = session.run(LOSS_,
                          feed_dict={X:x,Y:y})
        #'''
        if i%1000==0:
            print(epoch,i,':',current_loss)


0 0 : 3.77953
0 1000 : 2.58427


# KERAS USELESS

In [45]:
from keras.models import Sequential
from keras.layers import Dense, Activation,TimeDistributed
from keras.layers import LSTM,LSTMCell
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

import random
import sys

In [ ]:
text = t0.sum()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 1
sentences = []
next_chars = []
for i in range(5):
    line = t0[i]
    l = len(line)
    sentence = line[: l-1]
    next_char = line[1: l]
    x = np.zeros((l, 1, len(chars)), dtype=np.bool)
    y = np.zeros((l,1, len(chars)), dtype=np.bool)
    for t,letter in enumerate(sentence):
        x[t, 0, char_indices[letter]] = 1
    for t,letter in enumerate(next_char):
        y[t, 0, char_indices[letter]] = 1
    sentences.append(x)
    next_chars.append(y)
print('nb sequences:', len(sentences))

print('Vectorization...')
'''
for i  in range(len(sentences)):
    line = sentences[i]
    l = len(line)
    
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
'''

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(None, len(chars))))
model.add((Dense(len(chars))))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(len(sentences)):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    x = sentences[iteration]
    y = next_chars[iteration]
    model.fit(x, y,
              batch_size=None,
              epochs=1,verbose=0)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()